In [1]:
import pyvo as vo
from astroquery.mast import Mast
from astropy.table import Table
import requests
import astropy
import numpy as np
import pylab
import time
import json
from matplotlib import pyplot as plt
from IPython.core.display import display
from IPython.core.display import HTML
import warnings
warnings.filterwarnings("ignore", module="astropy.io.votable.*")
warnings.filterwarnings("ignore", module="pyvo.utils.xml.elements")

/tmp/ipykernel_3501/1231743142.py:11: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display


In [30]:
import pyvo as vo
import pandas as pd
import numpy as np
import os
from astropy.time import Time
from astropy import units as u
from tqdm import tqdm, trange
from math import ceil
from astropy.time import TimeDelta
import time
import threading
import multiprocessing

In [2]:
TAP_service = vo.dal.TAPService("https://vao.stsci.edu/PS1DR2/tapservice.aspx")

In [24]:
query = """SELECT
objID, detectID, Detection.filterID as filterID, Filter.filterType, obsTime, ra, dec,
psfFlux, psfFluxErr, psfMajorFWHM, psfMinorFWHM, psfQfPerfect, 
apFlux, apFluxErr, infoFlag, infoFlag2, infoFlag3
FROM Detection
NATURAL JOIN Filter
WHERE objID >= 72002759236154323
AND objID <= 72003271819814312
ORDER BY filterID, obsTime
"""
print(query)

job = TAP_service.search(query)
detection_TAP_results = job.to_table()
detection_TAP_results

SELECT
objID, detectID, Detection.filterID as filterID, Filter.filterType, obsTime, ra, dec,
psfFlux, psfFluxErr, psfMajorFWHM, psfMinorFWHM, psfQfPerfect, 
apFlux, apFluxErr, infoFlag, infoFlag2, infoFlag3
FROM Detection
NATURAL JOIN Filter
WHERE objID >= 72002759236154323
AND objID <= 72003271819814312
ORDER BY filterID, obsTime



objID,detectID,filterid,filterType,obsTime,ra,dec,psfFlux,psfFluxErr,psfMajorFWHM,psfMinorFWHM,psfQfPerfect,apFlux,apFluxErr,infoFlag,infoFlag2,infoFlag3
,,,,,deg,deg,,,,,,,,,,
int64,int64,uint8,object,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,int64,int32,int32
72002801748879088,89850505040007306,1,g,54999.5054092,280.17486597,-29.99269896,4.74451e-05,1.25481e-05,2.36486,1.86551,0.446414,2.81233e-05,1.07269e-06,35651585,0,3145728
72002801754357527,89850505040008502,1,g,54999.5054092,280.17541498,-29.99399945,7.40373e-05,1.23287e-05,2.36486,1.86551,0.477376,1.21513e-05,7.11918e-07,35651585,0,3145728
72002801782721082,89850505040002367,1,g,54999.5054092,280.17825437,-29.99937001,0.000319424,1.21124e-05,2.36487,1.86552,0.517788,0.000330719,3.71409e-06,102760453,128,0
72002801785808280,89850505040004123,1,g,54999.5054092,280.17857231,-29.99341892,0.000208433,8.23337e-06,2.3649,1.86555,0.999082,0.000131464,2.31922e-06,35651585,128,7342144
72002801804842373,89850505040007902,1,g,54999.5054092,280.18048,-29.99830059,0.000131698,7.87816e-06,2.3649,1.86555,0.999321,0.00010994,2.12507e-06,35651585,0,7342144
72002801808126490,89850505040009835,1,g,54999.5054092,280.18079823,-29.99488572,0.000141958,7.87803e-06,2.36493,1.86557,0.999412,0.000203319,2.89457e-06,35667969,0,124782656
72002801826034495,89850505040003082,1,g,54999.5054092,280.18259505,-29.99651777,0.000277004,8.4218e-06,2.36494,1.86558,0.999422,0.000233402,3.10447e-06,102760453,128,7342144
72002801829676324,89850505040004945,1,g,54999.5054092,280.18292553,-29.99497545,0.000158017,7.92758e-06,2.36496,1.86559,0.999409,0.000186014,2.76864e-06,35651585,0,7342144


In [22]:
query = """ Select objID FROM dbo.MeanObjectView Where objID > 72002759234365113 AND 
"""
print(query)

job = TAP_service.search(query)
detection_TAP_results = job.to_table()
detection_TAP_results

 Select objID FROM dbo.MeanObjectView Where objID > 72002759234365113



objID
int64
72002759236154323
72002759237522000
72002759237750310
72002759240179723
72002759242908506
72002759243332517
72002759244803110
72002759248576572
72002759253587362


In [27]:
query = """ Select Min(objID) FROM dbo.MeanObjectView """
print(query)

job = TAP_service.search(query)
detection_TAP_results = job.to_table()
detection_TAP_results

 Select Min(objID) FROM dbo.MeanObjectView 


DALQueryError: Query execution problem: Execution Timeout Expired.  The timeout period elapsed prior to completion of the operation or the server is not responding.

In [ ]:
a = 0
number = 10
DATA_DIR = "/epyc/projects/adam_datasets/ps1_dr2/objects/"

while(number > 0):
    query = f""" Select objID FROM dbo.MeanObjectView Where objID > {a} AND nDetections < 4"""
    tap_results = TAP_service.search(query)
    df = pd.DataFrame(tap_results) 
    number = len(df)
    a = df['objID'].iloc[-1]
    df.to_csv(DATA_DIR + f"objID_{a}.csv",index = False)